<a href="https://colab.research.google.com/github/mp11-rohan/My-First-Website/blob/main/Sistemas_Recomendadores_Experiencia_Matem%C3%A1tica_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install scikit-surprise


In [5]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load the MovieLens dataset
!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

--2024-07-02 12:14:51--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   939KB/s    in 1.0s    

2024-07-02 12:14:52 (939 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [6]:
# Read the dataset into a pandas dataframe
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

# Prepare the data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [7]:
# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Calculate RMSE
print("RMSE: ", rmse(predictions))

RMSE: 0.8778
RMSE:  0.8777535417493472


In [8]:
# Function to get a list of movie ids that the user has not rated
def get_unrated_movies(user_id, ratings_df, movies_df):
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId']
    all_movies = set(movies_df['movieId'])
    rated_movies_set = set(rated_movies)
    unrated_movies = list(all_movies - rated_movies_set)
    return unrated_movies

In [9]:
# Function to recommend movies for a user
def recommend_movies(user_id, ratings_df, movies_df, algo, top_n=10):
    unrated_movies = get_unrated_movies(user_id, ratings_df, movies_df)
    predictions = [algo.predict(user_id, movie_id) for movie_id in unrated_movies]
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    recommended_movie_ids = [pred.iid for pred in top_predictions]
    recommended_movies = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]
    return recommended_movies

In [10]:
# Get recommendations for a specific user
user_id = 1
recommended_movies = recommend_movies(user_id, ratings, movies, algo, top_n=10)
print(f"Top 10 movie recommendations for user {user_id}:")
print(recommended_movies[['title', 'genres']])

Top 10 movie recommendations for user 1:
                                                 title  \
277                   Shawshank Redemption, The (1994)   
474                                Blade Runner (1982)   
596         Ghost in the Shell (Kôkaku kidôtai) (1995)   
602  Dr. Strangelove or: How I Learned to Stop Worr...   
681                         Singin' in the Rain (1952)   
690                          North by Northwest (1959)   
694                                  Casablanca (1942)   
696                                My Fair Lady (1964)   
704      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   
868        Wallace & Gromit: The Wrong Trousers (1993)   

                                        genres  
277                                Crime|Drama  
474                     Action|Sci-Fi|Thriller  
596                           Animation|Sci-Fi  
602                                 Comedy|War  
681                     Comedy|Musical|Romance  
690  Action|Adventure|Myst